In [1]:
from scipy.io import loadmat, savemat
import numpy as np
import time
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn import cross_validation

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
from keras import backend as K

from keras.layers import Input, Dense, Dropout, merge, concatenate, Convolution3D, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Lambda
from keras.models import Model
from keras.models import Sequential
from keras.layers import Activation
from keras.wrappers.scikit_learn import KerasRegressor

from keras.optimizers import SGD, adam, nadam, Adagrad, RMSprop
from keras.regularizers import l1,l2

from keras.callbacks import EarlyStopping, CSVLogger


import os
import os.path
import sys
import argparse
import time
import csv

D:\Users\Vishwesh\Anaconda3\envs\deep_l\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12523498828434716435
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 5080789811
locality {
  bus_id: 1
}
incarnation: 13176037114117543784
physical_device_desc: "device: 0, name: GeForce GTX 970M, pci bus id: 0000:01:00.0"
]


In [3]:
def euclidean_distance(vects):
    x, y, z = vects
    #euc_dist = K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))    
    #euc_dist = K.square(x - y)
    euc_dist = K.square(x - y)
    out = euc_dist + z
    #print(euc_dist)
    #print(out)
    return out

In [4]:
def identity_loss(y_true, y_pred):
    #loss_1 = K.cast(loss_1, dtype='float64')
    
    # Extract aux1, aux2 and main o/p
    loss_1 = K.mean(K.square(y_pred[:,:66]-y_pred[:,66:132]))
    loss_2 = K.mean(K.square(y_pred[:,132:]-y_true))
    loss = loss_1 + loss_2
    #loss_1 = K.mean(y_true - y_pred)
    #return K.mean(y_pred - 0 * y_true)
    return loss

In [5]:
def create_cnn_network(input_dim):
    
    seq = Sequential()
    
    seq.add(Dense(45, input_shape=(input_dim,)))
    #seq.add(BatchNormalization())
    
    seq.add(Dense(400))
    seq.add(Activation("relu"))
    #seq.add(BatchNormalization())
    
    seq.add(Dense(66))
    seq.add(Activation("relu"))
    #seq.add(BatchNormalization())
    
    seq.add(Dense(200))
    #seq.add(BatchNormalization())
    
    # Output Layer.
    seq.add(Dense(66))
    print(seq.summary())
    return seq

In [6]:
def load_anchor():
    # Load all data
    ip = loadmat('train_anchor_input.mat')
    main_X = np.array(ip['train_anchor_input'])
    
    output = loadmat('train_anchor_output.mat')
    y = np.array(output['train_anchor_output'])
    
    # Get dimensions of arrays
    x_size = main_X.shape
    print('Input Array Shape',x_size)
    y_size = y.shape
    print ('Output Array Shape',y_size)
    
    return main_X,y

In [7]:
def load_aux_data():
    aux_1 = loadmat('ta_voxels.mat')
    aux_2 = loadmat('tb_voxels.mat')

    aux_1_ip = np.array(aux_1['ta_voxels'])
    aux_2_ip = np.array(aux_2['tb_voxels'])

    aux_1_size = aux_1_ip.shape
    print ('Aux 1 Array Shape',aux_1_size)
    aux_2_size = aux_2_ip.shape
    print ('Aux 2 Array Shape',aux_2_size)
    
    return aux_1_ip,aux_2_ip


In [8]:
def build_nsnet():
    input_dim = 45
    # Three inputs
    input_a = Input(shape=(input_dim,))
    input_b = Input(shape=(input_dim,))
    main_ip = Input(shape=(input_dim,))
    
    # Create the base net structure
    base_network = create_cnn_network(input_dim)

    # Feed the inputs to the base network
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)
    processed_main = base_network(main_ip)
    
    # Combine the pairwise voxels with the histo voxel.
    #distance = Lambda(euclidean_distance)([processed_a, processed_b, processed_main])
    concat_layer = concatenate([processed_a,processed_b,processed_main])
    model = Model(input=[input_a, input_b, main_ip], output=concat_layer)
    
    opt_func = RMSprop()
    #model.compile(loss='mse', optimizer=opt_func)
    model.compile(loss=identity_loss, optimizer=opt_func)
    print(model.summary())
    return model


In [9]:
def train_nsnet(model, aux_1_ip, aux_2_ip, main_X, y, out_dir, n_epoch, val_size):
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    csv_logger = CSVLogger(os.path.join(out_dir, 'results.csv'))
    model.fit([aux_1_ip, aux_2_ip, main_X], y, epochs=n_epoch, batch_size=100, verbose=1, shuffle=True, validation_split=val_size, callbacks=[csv_logger])
    return model

In [10]:
def save_estimate(model, aux_1_ip, aux_2_ip, X, y, out_file, indices):
        
    pred = model.predict([aux_1_ip, aux_2_ip, X])
    
    output_test = loadmat('test_set_output_10th_order_final.mat')
    print(pred.shape)
    
    out_path = os.path.dirname(out_file)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
     
    savemat(out_file, mdict={'out_pred': pred, 'out_true': y})
    

In [11]:
def save_estimate_blind_hist(model, out_file):
        
    # Load the testing data
    input_test = loadmat('test_set_input_b6000.mat')
    input_dummy_a = loadmat('dummy_a.mat')
    input_dummy_b = loadmat('dummy_b.mat')
    
    # Make numpy arrays
    X_f_t = np.array(input_test['test_set_input_b6000'])
    X_f_a = np.array(input_dummy_a['dummy_a'])
    X_f_b = np.array(input_dummy_b['dummy_b'])
    
    pred = model.predict([X_f_a, X_f_b, X_f_t])
    
    output_test = loadmat('test_set_output_10th_order_final.mat')
    y_f_t = np.array(output_test['test_set_output_10th_order_final'])
    print(pred.shape)
    
    out_path = os.path.dirname(out_file)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
     
    savemat(out_file, mdict={'out_pred': pred, 'out_true': y_f_t})

In [12]:
def save_estimate_ts04(model, out_file_1, out_file_2):
    # In vivo test
    ts4_3ta = loadmat('sh_ts04_3ta_feed.mat')
    ts4_3tb = loadmat('sh_ts04_3tb_feed.mat')

    # Make numpy arrays
    X_3ta = np.array(ts4_3ta['sh_ts04_3ta_feed'])
    X_3tb = np.array(ts4_3tb['sh_ts04_3tb_feed'])

    input_dummy_a = loadmat('in_vivo_dummy_a.mat')
    input_dummy_b = loadmat('in_vivo_dummy_b.mat')

    X_f_a = np.array(input_dummy_a['dummy_a'])
    X_f_b = np.array(input_dummy_b['dummy_b'])
    
    
    out_path = os.path.dirname(out_file_1)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    
    # Pred 3TA TS04
    pred = model.predict([X_f_a,X_f_b,X_3ta])
    savemat(out_file_1, mdict={'predicted':pred})
    print('TS04 f1 Saved')
    
    # Pred 3TB TS04
    predi = model.predict([X_f_a,X_f_b,X_3tb])
    savemat(out_file_2, mdict={'predicted':predi})
    print('TS04 f2 Saved')

In [13]:
def save_estimate_ts01(model, out_file_1, out_file_2):
    # In vivo test
    ts4_3ta = loadmat('sh_ts01_3tb_feed.mat')
    ts4_3tb = loadmat('sh_ts01_austin_feed.mat')

    # Make numpy arrays
    X_3ta = np.array(ts4_3ta['sh_ts01_3tb_feed'])
    X_3tb = np.array(ts4_3tb['sh_ts01_austin_feed'])

    input_dummy_a = loadmat('in_vivo_dummy_ts01_a.mat')
    input_dummy_b = loadmat('in_vivo_dummy_ts01_b.mat')

    X_f_a = np.array(input_dummy_a['dummy_a'])
    X_f_b = np.array(input_dummy_b['dummy_b'])
    
    out_path = os.path.dirname(out_file_1)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    
    # Pred 3TB TS01
    pred = model.predict([X_f_a,X_f_b,X_3ta])
    savemat(out_file_1, mdict={'predicted':pred})
    print('TS01 f1 Saved')
    
    # Pred Austin TS01
    predi = model.predict([X_f_a,X_f_b,X_3tb])
    savemat(out_file_2, mdict={'predicted':predi})
    print('TS01 f2 Saved')

In [14]:
def main():
    work_dir = os.getcwd()
    exp = 'Null_space_CV_MICCAI_extended_loss_fixed_final'
    itr = 3 # Estimated from CV
    
    print('Loading Data ...')
    X, y = load_anchor()
    indices = np.array(range(X.shape[0]))+1
    
    aux_1_ip, aux_2_ip = load_aux_data()
    
    out_start_dir = os.path.join(work_dir,exp)
    seed1 = 46
    
    kf = KFold(n_splits=5, random_state=seed1,shuffle=True)
    
    model_D = build_nsnet()
    fold_num = 0
    
    # Size of split for training and validation
    val_size=0.2
    
    for train, test in kf.split(X):
        fold_num += 1
        X_train = X[train,:]
        aux_1_ip_train = aux_1_ip[train,:]
        aux_2_ip_train = aux_2_ip[train,:]
        y_train = y[train,:]
        
        X_test = X[test,:]
        aux_1_ip_test = aux_1_ip[test,:]
        aux_2_ip_test = aux_2_ip[test,:]
        y_test = y[test,:]
        
        
        
        indices_train = indices[train]
        indices_test = indices[test]
        
        print ("Training DNN with %d iterations, fold %d" % (itr, fold_num))
        out_dir_DNN = os.path.join(out_start_dir, str(fold_num))
        model_D = train_nsnet(model_D, aux_1_ip_train, aux_2_ip_train, X_train, y_train, out_dir_DNN, itr, val_size)
        
        print ("Saving training outputs")
        end_dir = os.path.join(out_start_dir, str(fold_num), 'training.csv')
        # Set all aux_ip_train while predictiing to zeros
        aux_1_ip_train.fill(0)
        aux_2_ip_train.fill(0)
        save_estimate(model_D, aux_1_ip_train, aux_2_ip_train, X_train, y_train, end_dir,indices_train)

        print ("Saving testing outputs")
        end_dir = os.path.join(out_start_dir, str(fold_num), 'testing.csv')
        # Set all aux_ip_tests to zeros
        aux_1_ip_test.fill(0)
        aux_2_ip_test.fill(0)
        save_estimate(model_D, aux_1_ip_test, aux_2_ip_test, X_test, y_test, end_dir,indices_test)
    
    print ("Saving Histology blind test outputs")
    end_dir = os.path.join(out_start_dir, str('Hist_Blind_72_Test'), 'result_NSDN_b6000_testing_10th_order.mat')
    save_estimate_blind_hist(model_D, end_dir)
    
    
    print ("Saving TS04 Reproducibility Results")
    end_dir_1 = os.path.join(out_start_dir, str('TS04'), 'result_NSDN_3ta.mat')
    end_dir_2 = os.path.join(out_start_dir, str('TS04'), 'result_NSDN_3tb.mat')
    save_estimate_ts04(model_D, end_dir_1, end_dir_2)
    
    print ("Saving TS01 Reproducibility Results")
    end_dir_3 = os.path.join(out_start_dir, str('TS01'), 'result_NSDN_3tb.mat')
    end_dir_4 = os.path.join(out_start_dir, str('TS01'), 'result_NSDN_austin.mat')
    save_estimate_ts01(model_D, end_dir_3, end_dir_4)
    
    


In [15]:
if __name__=='__main__':
    main()

Loading Data ...
Input Array Shape (34679, 45)
Output Array Shape (34679, 66)
Aux 1 Array Shape (34679, 45)
Aux 2 Array Shape (34679, 45)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 45)                2070      
_________________________________________________________________
dense_2 (Dense)              (None, 400)               18400     
_________________________________________________________________
activation_1 (Activation)    (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 66)                26466     
_________________________________________________________________
activation_2 (Activation)    (None, 66)                0         

D:\Users\Vishwesh\Anaconda3\envs\deep_l\lib\site-packages\ipykernel\__main__.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("co...)`



_________________________________________________________________
dense_4 (Dense)              (None, 200)               13400     
_________________________________________________________________
dense_5 (Dense)              (None, 66)                13266     
Total params: 73,602
Trainable params: 73,602
Non-trainable params: 0
_________________________________________________________________
None
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 45)            0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 45)            0                                            
_________________________________________________________________________________________